In [1]:
import os
from IPython import display
from matplotlib import pyplot as plt
import rasterio
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
import numpy as np
import pandas as pd
import sklearn
import math
from zipfile import *
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, accuracy_score

C:\Users\ipdavies\AppData\Local\Continuum\miniconda3\envs\thesis\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ipdavies\AppData\Local\Continuum\miniconda3\envs\thesis\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ipdavies\AppData\Local\Continuum\miniconda3\envs\thesis\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", n

In [40]:
import sys
sys.path.append('../')
from CPR.configs import data_path
from CPR.utils import tif_stacker, preprocessing, train_val, cloud_generator

In [18]:
# Order in which features should be stacked to create stacked tif
feat_list_new = ['aspect','curve', 'developed', 'GSW_distExtent', 'elevation', 'forest',
 'GSW_maxExtent', 'hand', 'other_landcover', 'planted', 'slope', 'spi', 'twi', 'wetlands', 'flooded']

img_list = ['4115_LC08_021033_20131227_test']

for j, img in enumerate(img_list):
    #  Stack all the flood imagery
    tif_stacker(path_data, img, feat_list_new, overwrite=False)
    cloud_generator(img, data_path, overwrite=False)

TypeError: unsupported operand type(s) for +: 'WindowsPath' and 'str'

In [5]:
pctl = [20]
data, data_ind = preprocessing(path, img, pctl)
data_vector, training_data, validation_data, training_size = train_val(data)

### Try using UncertaintyNN
from [here](https://github.com/hutec/UncertaintyNN)

## Try creating NN with MCD in tf.keras

In [254]:
X_train, y_train = data_vector[:,0:14], data_vector[:,14]
X_val, y_val = validation_data[:,0:14], validation_data[:,14]

# # Convert labels to one-hot encoding
# y_train = tf.keras.utils.to_categorical(y_train)
# y_val = tf.keras.utils.to_categorical(y_val)

# data_vector_expand = np.expand_dims(data_vector, axis=-1)
# validation_data_expand = np.expand_dims(validation_data, axis=-1)

# X_train, y_train = data_vector_expand[:,0:14], data_vector_expand[:,14]
# X_val, y_val = validation_data_expand[:,0:14], validation_data_expand[:,14]

pctl = 20
batch_size = 100
epochs = 5
input_dims = X_train.shape[1]
dropout_rate = 0.3
mc_passes = 100

In [258]:
def get_NN_MCD():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(input_dims,))),
    model.add(tf.keras.layers.Lambda(lambda x: K.dropout(x, level=0.3))),
    model.add(tf.keras.layers.Dense(input_shape=(input_dims,),
                                    units=28,
                                    activation='relu')),
    model.add(tf.keras.layers.Lambda(lambda x: K.dropout(x, level=0.3))),
    model.add(tf.keras.layers.Dense(units=12,
                                    activation='relu')),
#     model.add(tf.keras.layers.Dropout(rate=dropout_rate)(training=True)),
#     model.add(tf.keras.layers.Flatten()),
    model.add(tf.keras.layers.Dense(2,
                                    activation='softmax'))
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',      
#                   metrics=[tf.keras.metrics.Recall()])
#                   metrics=[tf.keras.metrics.F1()])
                  metrics=['accuracy'])                  
    return model

NN_MCD = get_NN_MCD()
NN_MCD.summary()

NN_MCD.fit(X_train, y_train,
          batch_size = batch_size,
          epochs= epochs,
          verbose = 1,
          validation_data = (X_val, y_val))

Model: "sequential_33"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_10 (Lambda)           (None, 14)                0         
_________________________________________________________________
dense_84 (Dense)             (None, 28)                420       
_________________________________________________________________
lambda_11 (Lambda)           (None, 28)                0         
_________________________________________________________________
dense_85 (Dense)             (None, 12)                348       
_________________________________________________________________
dense_86 (Dense)             (None, 2)                 26        
Total params: 794
Trainable params: 794
Non-trainable params: 0
_________________________________________________________________
Train on 316099 samples, validate on 31609 samples
Epoch 1/5
316099/316099 [==============================] - 15s 47us/sa

In [40]:
pctl = [20]
data_vector_gaps, data_ind_gaps = preprocessing_gaps(path, img, pctl)
X_test, y_test = data_vector_gaps[:,0:14], data_vector_gaps[:,14]

In [249]:
mc_passes = 20
def predict_with_uncertainty(model, X):
    preds = []
    for i in range(mc_passes):
        preds.append(model.predict(X))
    preds = np.array(preds)
    means = np.mean(preds, axis=0)
    variances = np.var(preds, axis=0)
    stds = np.std(preds, axis=0)
    pred = np.argmax(means, axis=1)
    
    return pred, preds, means, variances, stds

pred, preds, means, variances, stds = predict_with_uncertainty(NN_MCD, X_test)

In [ ]:
# Plotting how many MC samples predicted flooding?
# (# times the prob of flooding > 0.5) / # samples
# The color bar should be more saturated in the middle to highlight greater
# uncertainty, and lighter towards 0% and 100%.

# flood = 0
# flood += 